In [1]:
import sys
import os
sys.path.append(os.path.expanduser('~/l/ocgis/src'))
sys.path.append(os.path.expanduser('~/l/esmf/src/addon/ESMPy/src'))
import ocgis
assert(ocgis.__release__ == '2.1.0.dev1')

Configure some environment variables to point to the head directory containing climate data files used in the demo as well as the output directory.

In [2]:
import tempfile
ocgis.env.DIR_DATA = os.path.expanduser('~/l/data/ocgis_test_data/CanCM4')
ocgis.env.DIR_OUTPUT = tempfile.mkdtemp()
print(ocgis.env.DIR_OUTPUT)

/tmp/tmpzz_amE


Inspect a target file's metadata.

In [3]:
uri = 'tas_day_CanCM4_decadal2000_r2i1p1_20010101-20101231.nc'
variable = 'tas'
rd = ocgis.RequestDataset(uri=uri, variable=variable)
rd.inspect()

OCGIS Driver Key: netcdf-cf {
dimensions:
    time = ISUNLIMITED ; // 3650 currently
    lat = 64 ;
    lon = 128 ;
    bnds = 2 ;
variables:
    float64 time(time) ;
      time:bounds = "time_bnds" ;
      time:units = "days since 1850-1-1" ;
      time:calendar = "365_day" ;
      time:axis = "T" ;
      time:long_name = "time" ;
      time:standard_name = "time" ;
    float64 time_bnds(time, bnds) ;
    float64 lat(lat) ;
      lat:bounds = "lat_bnds" ;
      lat:units = "degrees_north" ;
      lat:axis = "Y" ;
      lat:long_name = "latitude" ;
      lat:standard_name = "latitude" ;
    float64 lat_bnds(lat, bnds) ;
    float64 lon(lon) ;
      lon:bounds = "lon_bnds" ;
      lon:units = "degrees_east" ;
      lon:axis = "X" ;
      lon:long_name = "longitude" ;
      lon:standard_name = "longitude" ;
    float64 lon_bnds(lon, bnds) ;
    float64 height() ;
      height:units = "m" ;
      height:axis = "Z" ;
      height:positive = "up" ;
      height:long_name = "height" ;
      

The dimension map defines how metadata is interpreted. This can be customized to deal with non-conforming data or special use cases.

In [4]:
rd.dimension_map.pprint()


{'crs': {'variable': 'latitude_longitude'},
 'driver': 'netcdf-cf',
 'level': {'attrs': {},
           'bounds': None,
           'dimension': [],
           'variable': u'height'},
 'realization': {'dimension': [], 'variable': None},
 'time': {'attrs': {'axis': 'T'},
          'bounds': u'time_bnds',
          'dimension': [u'time'],
          'variable': u'time'},
 'x': {'attrs': {},
       'bounds': u'lon_bnds',
       'dimension': [u'lon'],
       'variable': u'lon'},
 'y': {'attrs': {},
       'bounds': u'lat_bnds',
       'dimension': [u'lat'],
       'variable': u'lat'}}


For unstructured data, it is possible to hold multiple geometric abstractions in the dimension map each defining their own metadata interpretation.

In [5]:
path = os.path.expanduser('~/l/i49-ugrid-cesm/UGRID_1km-merge-10min_HYDRO1K-merge-nomask_c130402.nc')
ocgis.RequestDataset(path, driver='netcdf-ugrid').dimension_map.pprint(as_dict=True)


{'attribute_host': {'attrs': {u'cf_role': u'mesh_topology',
                              u'dimension': 2,
                              u'face_coordinates': u'landmesh_face_x landmesh_face_y',
                              u'face_node_connectivity': u'landmesh_face_node',
                              u'locations': u'face node',
                              u'node_coordinates': u'landmesh_node_x landmesh_node_y',
                              u'standard_name': u'mesh_topology'},
                    'bounds': None,
                    'dimension': [],
                    'variable': u'landmesh'},
 'crs': {'variable': 'latitude_longitude'},
 'driver': 'netcdf-ugrid',
 'level': {'dimension': [], 'variable': None},
 'realization': {'dimension': [], 'variable': None},
 'time': {'dimension': [], 'variable': None},
 'topology': {'point': {'x': {'attrs': {},
                              'bounds': None,
                              'dimension': [u'nlandmesh_face'],
                        

Subset a target file by the boundary of California using an intersects GIS operation (the default), and write the data to an ESRI Shapefile. Select the first time coordinate only. If scripted, it may be executed with `mpirun` or equivalent for a parallel run.

In [6]:
geom = os.path.expanduser('~/l/shp/state_boundaries/state_boundaries.shp')
ops = ocgis.OcgOperations(dataset=rd, geom=geom, geom_select_uid=[25], output_format='shp', prefix='ca', 
                          snippet=True)
ca_shp = ops.execute()
print(ca_shp)

/tmp/tmpzz_amE/ca/ca.shp


 Also write the model grid to shapefile.

In [ ]:
ops = ocgis.OcgOperations(dataset=rd, output_format='shp', snippet=True, prefix='grid', vector_wrap=True)
ca_grid = ops.execute()
print(ca_grid)

As a brief aside, data "payloads" or only loaded when needed. For example, in this CA subset, there is no data variable values loaded until they are requested for conversion or calculations.

In [ ]:
field = rd.create_field()
assert(field.data_variables[0]._value is None)

Spatially average the grid cells clipped to the boundary of California for all the June, July, and August months in the target dataset. Write the output data to CSV.

In [ ]:
import webbrowser
rd = ocgis.RequestDataset(uri=uri, variable=variable, time_region={'month': [6, 7, 8]})
ops = ocgis.OcgOperations(dataset=rd, geom=geom, geom_select_uid=[25], spatial_operation='clip',
                          output_format='csv', prefix='ca_spatial_average', aggregate=True)
ret = ops.execute()
print(ret)
webbrowser.open(ret)

Perform a difference calulation between two variables using a string function. Inspect the metadata of the output NetCDF file.

In [ ]:
rd1 = ocgis.RequestDataset(uri='tasmax_day_CanCM4_decadal2000_r2i1p1_20010101-20101231.nc',
                           variable='tasmax')
rd2 = ocgis.RequestDataset(uri='tasmin_day_CanCM4_decadal2000_r2i1p1_20010101-20101231.nc',
                           variable='tasmin')
calc = 'diff=tasmax-tasmin'
ops = ocgis.OcgOperations(dataset=[rd1, rd2], calc=calc, output_format='nc', geom='state_boundaries',
                          select_ugid=[25], prefix='diff')
ret = ops.execute()
ocgis.RequestDataset(ret).inspect()

Calculate a sequence of statistics to produce a July time series conforming the target units from Kelvin to Celsius. Perform the calculations on the spatially averaged data for California.

In [ ]:
import webbrowser
rd = ocgis.RequestDataset(uri=uri, variable=variable, time_region={'month': [7]}, conform_units_to='celsius', 
                          field_name='calcs')
calc = [{'func': 'mean', 'name': 'mean'},
        {'func': 'std', 'name': 'stdev'},
        {'func': 'min', 'name': 'min'},
        {'func': 'max', 'name': 'max'},
        {'func': 'median', 'name': 'median'},
        {'func': 'freq_perc', 'name': 'fp_95', 'kwds': {'percentile': 95.0}},
        {'func': 'freq_perc', 'name': 'fp_5', 'kwds':{'percentile': 5.0}},]
calc_grouping = ['month','year']
ops = ocgis.OcgOperations(dataset=rd, geom='state_boundaries', geom_select_uid=[25, 26], spatial_operation='clip',
                          output_format= 'csv', prefix='ca_calcs', aggregate=True, calc=calc,
                          calc_grouping=calc_grouping)
ret = ops.execute()
print(ret)
webbrowser.open(ret)

Perform the same operation returning the data as a "collection". Print the derived variable aliases.

In [ ]:
ops.output_format = 'ocgis'
ret = ops.execute()
print(ret)
print(ret[25].groups[rd.field_name].keys())

Fields are sliceable by dimensions (variables have dimensions similar to NetCDF).

In [ ]:
mean = ret.get_element(variable_name='mean')
print('mean dimensions = {}\n'.format(mean.dimensions))
print('mean shape = {}\n'.format(mean.shape))
print('field shapes = {}\n'.format(mean.parent.shapes))
sub = mean.parent[{'time': slice(0, 1)}]
print('sliced field shapes = {}'.format(sub.shapes))

Print some time values from the time variable.

In [ ]:
field = mean.parent
time = field.time
print(time.value_numtime)
print(time.value_datetime)

Print example variable values.

In [ ]:
mean.get_masked_value().squeeze()

Geometries are stored as Shapely objects with associated attributes.

In [ ]:
print(type(ret.geoms[25]))
print(ret.geoms[25]).bounds
print(ret[25]['STATE_NAME'].get_value())[0]

OCGIS supports arbitrary parallel decompositions for variables with support for empty objects. The VM implements basic parallel operations like broadcast, scatter, gather, etc. This example simulates the creation of dimensions when running on four cores.

In [ ]:
newdist = ocgis.vmachine.mpi.OcgDist(size=4)
newdist.create_dimension(name='not_dist', size=100)
newdist.create_dimension(name='im_dist', size=6, dist=True)
newdist.update_dimension_bounds()

for simrank in range(4):
    print('rank = {}'.format(simrank))
    localdistdim = newdist.get_dimension('im_dist', rank=simrank)
    repdim = newdist.get_dimension('not_dist', rank=simrank)
    print(localdistdim.name, localdistdim.bounds_local)
    print(repdim.name, repdim.bounds_local)
    print('')

For three variables, calculate monthly averages for the year 2005 for each U.S. state boundary.

In [ ]:
rd1 = ocgis.RequestDataset(uri='tas_day_CanCM4_decadal2000_r2i1p1_20010101-20101231.nc')
rd2 = ocgis.RequestDataset(uri='tasmin_day_CanCM4_decadal2000_r2i1p1_20010101-20101231.nc')
rd3 = ocgis.RequestDataset(uri='tasmax_day_CanCM4_decadal2000_r2i1p1_20010101-20101231.nc')
calc = [{'func': 'mean', 'name': 'mean'}]
calc_grouping = ['month']
ops = ocgis.OcgOperations(dataset=[rd1, rd2, rd3], geom='state_boundaries', aggregate=True,
                          output_format='shp', spatial_operation='clip', prefix='temps',
                          calc=calc, calc_grouping=calc_grouping, time_region={'year': [2005]},
                          conform_units_to='fahrenheit')
ret = ops.execute()
print(ret)

Use ESMF regridding with a subset and spatial aggregation, writing the data to shapefile.

In [ ]:
rd_src = ocgis.RequestDataset(uri='tas_day_CanCM4_decadal2010_r2i1p1_20110101-20201231.nc',
                              variable='tas')
rd_dest = ocgis.RequestDataset(uri='nldas_met_update.obs.daily.pr.1991.nc')
regrid_options = {'with_corners': False}
ops = ocgis.OcgOperations(dataset=rd_src, regrid_destination=rd_dest, geom_select_uid=[6, 16], 
                          agg_selection=True, geom='state_boundaries', snippet=True,
                          output_format='shp', prefix='regrid', regrid_options=regrid_options)
print ops.execute()